In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import numpy as np
from tqdm import tqdm


# 加载数据
file_path = '../datasets_FIX2/FIX2_deduplicated_mangoNews_Nums3000p_CategoryMerge_new_undersampled_Example.csv'
# file_path = '../datasets_FIX2/FIX2_deduplicated_mangoNews_Nums3000p_CategoryMerge_new_undersampled.csv'

data = pd.read_csv(file_path,low_memory=False,lineterminator="\n")

# 加载BERT tokenizer和模型
model_name = '../bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)



/opt/conda/envs/default/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data.head()

,id,website_id,request_url,response_url,category1,category2,title,abstract,body,pub_time,cole_time,images,language_id,md5\r
0,17167183,2270,https://dmpnews.org/%e0%a6%a1%e0%a6%bf%e0%a6%8...,https://dmpnews.org/%e0%a6%a1%e0%a6%bf%e0%a6%8...,আইন,NaN,ডিএমপিতে পুলিশ পরিদর্শক পদমর্যাদার ৪ কর্মকর্তা...,ডিএমপিনিউজঃ ঢাকা মেট্রোপলিটন পুলিশের (ডিএমপি) ...,ডিএমপিনিউজঃ ঢাকা মেট্রোপলিটন পুলিশের (ডিএমপি) ...,21/6/2021 19:01:56,22/9/2023 02:11:22,"[""https://dmpnews.org/wp-content/uploads/2021/...",1779,a6c3ebaeff147b688325d6f9d9e35e66\r
1,18481478,2277,https://www.dhakatimes24.com/2023/02/25/300121,https://www.dhakatimes24.com/2023/02/25/300121,আইন,NaN,রাজধানীতে গোয়েন্দা পুলিশ পরিচয়ে যুবককে তুলে নে...,গোয়েন্দা পুলিশ (ডিবি) পরিচয়ে যুবককে তুলে নিয়ে ...,গোয়েন্দা পুলিশ (ডিবি) পরিচয়ে যুবককে তুলে নিয়ে ...,25/2/2023 17:57:00,29/9/2023 15:38:06,"[""https://www.dhakatimes24.com/assets/news_pho...",1779,0700bb5140c68658211786cb9b750820\r
2,18979479,2277,https://www.dhakatimes24.com/2019/02/12/112550,https://www.dhakatimes24.com/2019/02/12/112550,রাজনীতি,NaN,খালেদার মুক্তি নিয়ে নেতাদের কণ্ঠে হতাশার সুর,দুর্নীতি মামলায় দণ্ডিত হয়ে এক বছর ধরে কারাগারে...,দুর্নীতি মামলায় দণ্ডিত হয়ে এক বছর ধরে কারাগারে...,12/2/2019 19:01:00,1/10/2023 12:01:29,"[""https://www.dhakatimes24.com/assets/news_pho...",1779,1c6c3f4e3a57d3fd682c1f19f6f46708\r
3,17141761,2281,https://www.jugantor.com/todays-paper/news/524...,https://www.jugantor.com/todays-paper/news/524...,রাজনীতি,NaN,জনসম্পৃক্ততা বাড়াতে মাঠে নামছে বিএনপি,জনসম্পৃক্তমূলক কর্মসূচি নিয়ে এবার মাঠে নামছে ব...,জনসম্পৃক্তমূলক কর্মসূচি নিয়ে এবার মাঠে নামছে ব...,26/2/2022 00:00:00,22/9/2023 01:04:13,"[""https://www.jugantor.com/assets/news_photos/...",1779,d4e1581ebabc5169508cb2516cddff55\r
4,16357680,2283,https://www.ekushey-tv.com/%E0%A6%AC%E0%A6%BF%...,https://www.ekushey-tv.com/%E0%A6%AC%E0%A6%BF%...,অন্যান্য,NaN,বিদায় ২০২০ স্বাগতম ২০২১,এসে গেলো ২০২১। নতুন বছরের শুভেচ্ছা সবার জন্যে।...,বিশিষ্ট অর্থনীতিবিদ ড. সেলিম জাহান কানাডা ও যু...,31/12/2020 10:27:00,17/9/2023 15:59:15,"[""https://www.ekushey-tv.com/media/imgAll/2020...",1779,ad458bc48e81c34a332155add8a7ddf8\r


In [2]:
# 将模型移动到GPU(如果可用)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_model.to(device)

# 定义数据集类
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])
        label = self.labels[index]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# 将孟加拉语类别转换为数字标签
label_map = {label: i for i, label in enumerate(data['category1'].unique())}
labels = data['category1'].map(label_map).tolist()

# 将数据划分为训练集和测试集
train_texts, test_texts, train_labels, test_labels = train_test_split(data['body'].tolist(), labels, test_size=0.2, random_state=42)
print("1")
# 创建数据集和数据加载器
train_dataset = NewsDataset(train_texts, train_labels, tokenizer)
test_dataset = NewsDataset(test_texts, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)
print("1")

# 定义LSTM分类器
class LSTMClassifier(nn.Module):
    def __init__(self, bert_model, hidden_size, num_classes, num_layers=1, bidirectional=True, dropout=0.5):
        super(LSTMClassifier, self).__init__()
        self.bert = bert_model
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(bert_model.config.hidden_size, hidden_size, num_layers, bidirectional=bidirectional, batch_first=True)
        self.fc = nn.Linear(hidden_size * (2 if bidirectional else 1), num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        lstm_output, _ = self.lstm(last_hidden_state)
        lstm_output = self.dropout(lstm_output[:, -1, :])
        logits = self.fc(lstm_output)
        return logits

# 创建LSTM分类器实例
hidden_size = 128
num_classes = len(label_map)
lstm_classifier = LSTMClassifier(bert_model, hidden_size, num_classes)
lstm_classifier.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm_classifier.parameters(), lr=2e-5)
print("1")

# 训练模型
num_epochs = 1
for epoch in range(num_epochs):
    lstm_classifier.train()
    train_loss = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for i, batch in enumerate(progress_bar):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        logits = lstm_classifier(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}')

# 在测试集上评估模型
lstm_classifier.eval()
test_preds = []
test_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        logits = lstm_classifier(input_ids, attention_mask)
        preds = torch.argmax(logits, dim=1)

        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# 将数字标签转换回孟加拉语类别
label_map_inv = {i: label for label, i in label_map.items()}
test_preds = [label_map_inv[i] for i in test_preds]
test_labels = [label_map_inv[i] for i in test_labels]

# 评估模型性能
print("LSTM Classifier:")
print(classification_report(test_labels, test_preds))

1
1
1


Epoch 1/1, Train Loss: 1.9794
LSTM Classifier:
              precision    recall  f1-score   support

    অন্যান্য       0.85      0.61      0.71        18
    অর্থনীতি       0.63      0.59      0.61        29
         আইন       0.79      0.70      0.75        27
    খেলাধুলা       0.96      1.00      0.98        23
     বিজ্ঞান       1.00      0.25      0.40        24
      বিনোদন       0.94      0.81      0.87        21
     রাজনীতি       0.57      0.96      0.72        25
  লাইফস্টাইল       0.42      0.65      0.51        17
      শিক্ষা       0.70      0.88      0.78        16

    accuracy                           0.71       200
   macro avg       0.76      0.72      0.70       200
weighted avg       0.77      0.71      0.70       200



In [ ]:
-=====================================================================

In [2]:
# 将模型移动到GPU(如果可用)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_model.to(device)

# 定义数据集类
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])
        label = self.labels[index]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# 将孟加拉语类别转换为数字标签
label_map = {label: i for i, label in enumerate(data['category1'].unique())}
labels = data['category1'].map(label_map).tolist()

# 将数据划分为训练集和测试集
train_texts, test_texts, train_labels, test_labels = train_test_split(data['body'].tolist(), labels, test_size=0.2, random_state=42)

# 创建数据集和数据加载器
train_dataset = NewsDataset(train_texts, train_labels, tokenizer)
test_dataset = NewsDataset(test_texts, test_labels, tokenizer)

batch_size = 8
gradient_accumulation_steps = 4

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [3]:

# 定义LSTM分类器
class LSTMClassifier(nn.Module):
    def __init__(self, bert_model, hidden_size, num_classes, num_layers=1, bidirectional=True, dropout=0.5):
        super(LSTMClassifier, self).__init__()
        self.bert = bert_model
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(bert_model.config.hidden_size, hidden_size, num_layers, bidirectional=bidirectional, batch_first=True)
        self.fc = nn.Linear(hidden_size * (2 if bidirectional else 1), num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        lstm_output, _ = self.lstm(last_hidden_state)
        lstm_output = self.dropout(lstm_output[:, -1, :])
        logits = self.fc(lstm_output)
        return logits

# 创建LSTM分类器实例
hidden_size = 128
num_classes = len(label_map)
lstm_classifier = LSTMClassifier(bert_model, hidden_size, num_classes)
lstm_classifier.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm_classifier.parameters(), lr=2e-5)



In [4]:

# 训练模型
num_epochs = 5
for epoch in range(num_epochs):
    lstm_classifier.train()
    train_loss = 0
    for i, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        logits = lstm_classifier(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss = loss / gradient_accumulation_steps
        loss.backward()

        if (i + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        train_loss += loss.item() * gradient_accumulation_steps

    train_loss /= len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}')


# 在测试集上评估模型
lstm_classifier.eval()
test_preds = []
test_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        logits = lstm_classifier(input_ids, attention_mask)
        preds = torch.argmax(logits, dim=1)

        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# 将数字标签转换回孟加拉语类别
label_map_inv = {i: label for label, i in label_map.items()}
test_preds = [label_map_inv[i] for i in test_preds]
test_labels = [label_map_inv[i] for i in test_labels]

# 评估模型性能
print("LSTM Classifier:")
print(classification_report(test_labels, test_preds))

Epoch 1/5, Train Loss: 1.9163
Epoch 2/5, Train Loss: 1.0519
Epoch 3/5, Train Loss: 0.6993
Epoch 4/5, Train Loss: 0.5112
Epoch 5/5, Train Loss: 0.4501
LSTM Classifier:
              precision    recall  f1-score   support

    অন্যান্য       0.93      0.72      0.81        18
    অর্থনীতি       0.77      0.69      0.73        29
         আইন       0.71      0.81      0.76        27
    খেলাধুলা       0.92      1.00      0.96        23
     বিজ্ঞান       0.82      0.75      0.78        24
      বিনোদন       0.83      0.95      0.89        21
     রাজনীতি       1.00      0.76      0.86        25
  লাইফস্টাইল       0.82      0.82      0.82        17
      শিক্ষা       0.64      0.88      0.74        16

    accuracy                           0.81       200
   macro avg       0.83      0.82      0.82       200
weighted avg       0.83      0.81      0.82       200



In [ ]:
# batchsize：16
# lr:2e-5
# epoch20
# undersampled:0.93